In [ ]:
import datetime
import requests
import time

VData=dict # Type

In [ ]:
USERNAME='NatNgs'
JWT=input('JWT (example: "Bearer xxxxxxxxx")')

In [ ]:
# Functions utils
LAST_TNSL_CALL=datetime.datetime.now(datetime.timezone.utc)
def callTournesol(path: str):
	global LAST_TNSL_CALL
	BASE_URL='https://api.tournesol.app/'
	wait=1-(datetime.datetime.now(datetime.timezone.utc)-LAST_TNSL_CALL).total_seconds()
	if wait > 0:
		time.sleep(wait)
	response = requests.get(BASE_URL + path, headers={
		'Authorization': JWT
	})
	LAST_TNSL_CALL = datetime.datetime.now(datetime.timezone.utc)
	return response.json()

def callTournesolMulti(path: str, args:str=None, start:int=0, end:int=0) -> list[VData]:
	LIMIT=1000
	URL=f'{path}?limit={LIMIT}' + (('&' + args) if args else '')
	offset=start
	rs = callTournesol(URL + f'&offset={offset}')
	if not 'count' in rs or not 'results' in rs:
		print('##### ERROR #####')
		print(URL)
		print(rs)
		exit(-1)
		return []

	total=rs['count']
	allRes:list[VData] = rs['results']
	print(f'{len(allRes)}/{total}', end=' ')

	while len(allRes) < total and (end <= 0 or offset < end):
		offset += LIMIT
		rs = callTournesol(URL + f'&offset={offset}')
		total=rs['count']
		allRes += rs['results']
		print(f'{len(allRes)}', end=' ')
	print()

	return allRes

def get(json:VData, default, *fields):
	for f in fields:
		if f in json:
			json = json[f]
			if json is None:
				return default
		else:
			return default
	return json


In [ ]:
# All tournesol videos having 1 or 2 contributors and positive score
def public2contrib():
	print('Extracting...')
	allRes:list = callTournesolMulti('polls/videos/recommendations/', 'exclude_compared_entities=false&unsafe=true', start=2000, end=30000) # &metadata%5Blanguage%5D=en
	allRes = [vdata for vdata in allRes if 
		    get(vdata, 0, 'collective_rating', 'n_contributors') in [2]
		and get(vdata, 0, 'collective_rating', 'n_comparisons') >= 3
		and get(vdata, 0, 'collective_rating', 'tournesol_score')/get(vdata, 0, 'collective_rating', 'n_contributors') >= 5
	]
	allRes.sort(key=lambda vdata: (
		round(get(vdata, 0, 'collective_rating', 'tournesol_score')),
		get(vdata, 0, 'collective_rating', 'n_comparisons')
	), reverse=True)
	authors:dict[str,list] = dict()
	for vdata in allRes:
		a = get(vdata, '???', 'entity', 'metadata', 'uploader')
		authors.setdefault(a, list()).append(vdata)

	print('# Tournesol missing contributors list')
	print()
	print('Here are extracted all the videos in English that may go publically recommended with only one more contributor on [Tournesol](https://tournesol.app).')
	print()
	print('Extraction date:', str(datetime.datetime.now())[:16])
	print()
	print('Found', len(allRes), 'videos by', len(authors), 'channels')
	print()

	years={get(vdata, '???', 'entity', 'metadata', 'publication_date')[0:4] for vdata in allRes}

	for y in sorted(years):
		print(f'\n## Year {y}')
		for vdata in allRes:
			if get(vdata, '???', 'entity', 'metadata', 'publication_date')[0:4] == y:
				tscore=get(vdata, 0, 'collective_rating', 'tournesol_score')
				tcmps=get(vdata, 0, 'collective_rating', 'n_comparisons')
				tctrs=get(vdata, 0, 'collective_rating', 'n_contributors')
				tchannel=get(vdata, '???', 'entity', 'metadata', 'uploader').replace('\\', '\\\\').replace('(', '\(').replace(')', '\)').replace('[', '\[').replace(']', '\]')
				tname=get(vdata, '???', 'entity', 'metadata', 'name').replace('\\', '\\\\').replace('(', '\(').replace(')', '\)').replace('[', '\[').replace(']', '\]')
				tdur=get(vdata, 0, 'entity', 'metadata', 'duration')
				tminutes=f"{int(tdur/60):2d}:{tdur%60:02d}" if tdur < 3600 else f"{int(tdur/3600)}:{int((tdur%3600)/60):02d}:{tdur%60:02d}"
				print(f"- {tscore:3.0f}🌻 ({tcmps} cmps / {tctrs} user{'s' if tctrs > 1 else ''}) \[{tminutes}\] {tchannel}: [{tname}](https://tournesol.app/entities/{vdata['entity']['uid']})")

public2contrib()